# SpQR application on a Higgs dataset


In this notebook we train the SpQR model on the dataset HIGGS.csv from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/HIGGS).

This notebook is divided essentially in five parts:

    1. Downloading and preparing (slicing) the dataset
    2. Plotting features (to get an initial idea)
    3. Initializing the model
    4. Fitting and evaluating performances
    5. Plotting results


## Package needed

In [36]:
! git clone https://github.com/MarcoRiggirello/diglm.git
! cat diglm/src/diglm.py

fatal: destination path 'diglm' already exists and is not an empty directory.
cat: diglm/diglm/diglm.py: No such file or directory


In [37]:
import sys
import os

py_file_location = "diglm/src"
sys.path.append(py_file_location)

In [38]:
import os
import time

import numpy as np
import pandas as pd
import seaborn as sns
import imageio
from PIL import Image

from download import download_file

## Downloading data

To download the dataset we use the function download_file in the \texttt{download.py} module: it will check if the dataset already exists in the current directory or download it from the internet.

In [39]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz'
file_path = os.path.join('../model_test/download/HIGGS.csv.gz')
download_file(url, file_path)

## Plotting features of the dataset

We use seaborn to display scatterplot matrices of the correlation between features in the dataset.

In [ ]:
t0 = time.time()

column_labels = ['label','lepton pT', 'lepton eta', 'lepton phi',
                 'missing energy magnitude', 'missing energy phi',
                 'jet 1 pt', 'jet 1 eta', 'jet 1 phi',
                 'jet 1 b-tag', 'jet 2 pt', 'jet 2 eta',
                 'jet 2 phi', 'jet 2 b-tag', 'jet 3 pt',
                 'jet 3 eta', 'jet 3 phi', 'jet 3 b-tag',
                 'jet 4 pt', 'jet 4 eta', 'jet 4 phi',
                 'jet 4 b-tag', 'm_jj', 'm_jjj','m_lv',
                 'm_jlv', 'm_bb', 'm_wbb', 'm_wwbb']
data = pd.read_csv(file_path,
                   header=0,
                   names=column_labels,
                   nrows=8192*16)
t1 = time.time()
print(f'Time for reading all the data ~ {(t1-t0):.0f} s')

# Creating a gif of  correlation scatter plots                                                                        
tuple_var = [(1, 7), (8, 14), (15, 21), (22, 28)]
grid_list = [sns.PairGrid(data.head(500),
                          hue='label',
                          vars=column_labels[init:end],
                          corner=True)
             for init, end in tuple_var]

fig_list = []
for i, grid in enumerate(grid_list):
    grid.map_diag(sns.kdeplot)
    grid.map_offdiag(sns.kdeplot)
    grid.add_legend()
    fig_name = f'sns_plot{i}.png'
    grid.savefig(fig_name)
    fig_list.append(imageio.imread(fig_name))

imageio.mimsave('sns.gif',
                fig_list,
                duration = 5)
im = Image.open('sns.gif')
im.show()

## Slicing data: train, validation, test

We slice the dataframe to obtain train and test samples.
Data appear to have mixed labels already, so that a simple slicing is sufficient to obtain good sampling.

There are 11 milions entries: the last 500k are used as tests, as in the [original paper](https://www.nature.com/articles/ncomms5308.pdf).

In [42]:
BATCH_SIZE = 1024
TRAIN_SIZE = BATCH_SIZE * 16
VAL_SIZE = BATCH_SIZE 
TEST_SIZE = BATCH_SIZE * 16

ft_train = data[column_labels[22:]].head(TRAIN_SIZE).astype('float32')
ft_val = data[column_labels[22:]].head(VAL_SIZE).astype('float32')
ft_test = data[column_labels[22:]].tail(TEST_SIZE).astype('float32')

l_train = data[column_labels[0]].head(TRAIN_SIZE).astype('int32')
l_val = data[column_labels[0]].head(VAL_SIZE).astype('int32')
l_test = data[column_labels[0]].tail(TEST_SIZE).astype('int32')

In [43]:
import tensorflow as tf

# Recursive function to batch the data
def batch(list_df, dataframe, batch_size):
  if len(dataframe.index) <= batch_size:
    list_df.append(tf.convert_to_tensor(dataframe))
    return list_df
  else:
    list_df.append(tf.convert_to_tensor(dataframe.head(batch_size)))
    dataframe = dataframe.iloc[batch_size:]
    return batch(list_df, dataframe, batch_size) 


# Batching the samples: train, val and test
l_train_batched_2 = batch([], l_train, BATCH_SIZE)
ft_train_batched = batch([], ft_train, BATCH_SIZE)
dict_train_batched = [{'features': ft_train_batched[i], 'labels': l_train_batched[i]}
              for i in range(len(ft_train_batched))]

l_val_batched = batch([], l_val, BATCH_SIZE)
ft_val_batched = batch([], ft_val, BATCH_SIZE)
dict_val_batched = [{'features': ft_val_batched[i], 'labels': l_val_batched[i]}
              for i in range(len(ft_val_batched))]

l_test_batched = batch([], l_test, BATCH_SIZE)
ft_test_batched = batch([], ft_test, BATCH_SIZE)
dict_test_batched = [{'features': ft_test_batched[i], 'labels': l_test_batched[i]}
              for i in range(len(ft_test_batched))]

## Building the DIGLM model

In the following block, we build our Deeply Invertible Generalized Linear Model (DIGLM) algorithm.
We will then be able to train the model on labelled data.

The steps to create and train the model are:

1. Create the desired transformed outputs from a simple distribution with the same dimensionality as the input data;
2. Initialize the NeuralSplineFlow bijector;
3. Initialize the DIGLM model;
4. Create the training step functions.


In [44]:
from tensorflow_probability import distributions as tfd
from tensorflow_probability import glm as tfglm
from tensorflow_probability import bijectors as tfb
from tensorflow.keras import metrics # metrics to evaluate model performances

from spqr import NeuralSplineFlow as NSF
from Diglm import DIGLM


neural_spline_flow = NSF(splits=4)
glm = tfglm.Bernoulli()
NUM_FEATURES = 7

diglm = DIGLM(neural_spline_flow, glm, NUM_FEATURES)

We define training functions to loop over train data.

In [45]:
@tf.function
def train_step(optimizer, target_sample):
    """
    Train step function for the diglm model. Implements the basic steps for computing
    and updating the trainable variables of the model. It also
    calculates the loss on training and validation samples.

    :param optimizer: Optimizer fro gradient minimization (or maximization).
    :type optimizer: A keras.optimizers object
    :param target_sample: dictonary of labels and features of data to train the model
    :type target_sample: dict

    """

    with tf.GradientTape() as tape:
        # calculating loss and its gradient of training data
        loss = -tf.reduce_mean(diglm.weighted_log_prob(target_sample))
    variables = tape.watched_variables()
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return loss

## Training

We go on training the algorithm on train dataset. We also check for the loss on validation samples

In [ ]:
accuracy = metrics.BinaryAccuracy()
LR = 1e-3
NUM_EPOCHS = 2

loss = 0
val_loss = 0
history = []

learning_rate = tf.Variable(LR, trainable=False)
optimizer = tf.keras.optimizers.Adam(learning_rate)

y_true = l_test_batched[0]
y_pred, var_pred, grad_pred = diglm(ft_test_batched[0])
accuracy.update_state(y_true,
                      y_pred)
print(f'accuracy before training = {accuracy.result().numpy():.3f}')

accuracy.reset_state()
for epoch in tf.range(NUM_EPOCHS):
    if epoch % 10 == 0:
        print(f"Epoch n. {epoch+1}. Loss = {loss} \t Val Loss = {val_loss}.")
    # Iteration on batch for training
    for batch in dict_train_batched:
        t_start = time.time() # to calculate the time of each iteration
        loss = train_step(optimizer, batch)
        val_loss = -tf.reduce_mean(diglm.weighted_log_prob(dict_val_batched[0]))
        t_stop = time.time()

        # Printing results
        print(f'{(t_stop - t_start):.0f} s \t loss = {loss} \t val_loss {val_loss}')
        history.append([loss, val_loss])

        # Accuracy estimate
        for j in range(len(l_test_batched)):
          y_true = l_test_batched[j]
          y_pred, var_pred, grad_pred = diglm(ft_test_batched[j])
          accuracy.update_state(y_true,
                                y_pred)
        print(f'accuracy after training = {accuracy.result().numpy():.3f}')
